In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from dateutil.tz import tzlocal

# caiman
import caiman as cm
from caiman.paths import caiman_datadir
from caiman.utils.utils import download_demo
from caiman.source_extraction.cnmf import params as params
from caiman.motion_correction import MotionCorrect
from caiman.source_extraction.cnmf import cnmf as cnmf

In [ ]:
# motion_correction parameter
dxy = (2., 2.)  # spatial resolution in x and y in (um per pixel)
# note the lower than usual spatial resolution here
max_shift_um = (12., 12.)  # maximum shift in um
patch_motion_um = (100., 100.)  # patch size for non-rigid correction in um
pw_rigid = True       # flag to select rigid vs pw_rigid motion correction
# maximum allowed rigid shift in pixels
max_shifts = [int(a/b) for a, b in zip(max_shift_um, dxy)]
# start a new patch for pw-rigid motion correction every x pixels
strides = tuple([int(a/b) for a, b in zip(patch_motion_um, dxy)])
# overlap between patches (size of patch in pixels: strides+overlaps)
overlaps = (24, 24)
# maximum deviation allowed for patch with respect to rigid shifts
max_deviation_rigid = 3

fnames = [os.path.join(
    caiman_datadir(), 
    'example_movies',
    'Sue_2x_3000_40_-46.tif'
)]
# estimates save path can be same or different from raw data path

save_path = os.path.join(
    caiman_datadir(), 
    'example_movies',
    'Sue_2x_3000_40_-46_CNMF_estimates.nwb'
)
# filename to be created or processed

# dataset dependent parameters
fr = 15.  # imaging rate in frames per second
decay_time = 0.4  # length of a typical transient in seconds

mc_dict = {
    'fnames': fnames,
    'fr': fr,
    'decay_time': decay_time,
    'dxy': dxy,
    'pw_rigid': pw_rigid,
    'max_shifts': max_shifts,
    'strides': strides,
    'overlaps': overlaps,
    'max_deviation_rigid': max_deviation_rigid,
    'border_nan': 'copy',
    #'var_name_hdf5': 'acquisition/TwoPhotonSeries'
    'var_name_hdf5': 'TwoPhotonSeries'
}

opts = params.CNMFParams(params_dict=mc_dict)

In [ ]:
opts

### motion correction

In [ ]:
c, dview, n_processes = cm.cluster.setup_cluster(
    backend='local', n_processes=None, single_thread=False)

In [ ]:
param = opts.get_group('motion')
# param['pw_rigid'] = False

mc = MotionCorrect(
    fnames, 
    dview=dview, 
    var_name_hdf5=opts.data['var_name_hdf5'], 
    **param
)
mc.motion_correct(save_movie=True)

In [ ]:
# %% MEMORY MAPPING
border_to_0 = 0 if mc.border_nan == 'copy' else mc.border_to_0
# you can include the boundaries of the FOV if you used the 'copy' option
# during motion correction, although be careful about the components near
# the boundaries

# memory map the file in order 'C'
fname_new = cm.save_memmap(
    mc.mmap_file, base_name='memmap_', order='C',
    border_to_0=border_to_0)  # exclude borders

# now load the file
Yr, dims, T = cm.load_memmap(fname_new)   # (3000, 28900)
images = np.array(np.reshape(Yr.T, [T] + list(dims), order='F'))

In [ ]:
print(dims, ' ', T, ' ', images.shape)

### CNMF

In [ ]:
decay_time = 0.4
p = 1
nb = 2
only_init = True           # has to be `False` when seeded CNMF is used
gSig = (7, 7)               # expected half size of neurons in pixels, very important for proper component detection
ssub = 1
tsub = 1
merge_thr = 0.85

# params object
opts_dict = {
    'decay_time': decay_time,
    'p': p,
    'nb': nb,
    'only_init': only_init,
    'gSig': gSig,
    'ssub': ssub,
    'tsub': tsub,
    'merge_thr': merge_thr
}

opts = params.CNMFParams(params_dict=opts_dict)

In [ ]:
# dims, T = cm.source_extraction.cnmf.utilities.get_file_size(fname, var_name_hdf5=var_name_hdf5)
# z = np.zeros(dims)
# dims = z[indices].shape
dims = (170, 170)
T = 3000
order = 'C'
shape_mov = (np.prod(dims), T)
fname_tot = cm.paths.memmap_frames_filename('aa', dims, T, order)
mmap_images = np.memmap(fname_tot, mode='w+', dtype=np.float32,
          shape=cm.mmapping.prepare_shape(shape_mov), order=order)

mmap_images = np.reshape(mmap_images.T, [T] + list(dims), order='F')
# mmap_images[:] = images[:]

In [ ]:
cnm = cnmf.CNMF(n_processes, params=opts, dview=dview)
cnm = cnm.fit(mmap_images)

In [ ]:
# %% plot contours of found components
Cn = cm.local_correlations(mmap_images, swap_dim=False)
Cn[np.isnan(Cn)] = 0
cnm.estimates.plot_contours(img=Cn)
plt.title('Contour plots of found components')

#%% save results in a separate file (just for demonstration purposes)
cnm.estimates.Cn = Cn
# cnm.save(fname_new[:-4]+'hdf5')
#cm.movie(Cn).save(fname_new[:-5]+'_Cn.tif'

### wrapperを使用する

In [ ]:
import sys
sys.path.append('..')
from wrappers.data_wrapper import *
from wrappers.caiman_wrapper import *

In [ ]:
arr = np.zeros((3, 300, 300))
arr.shape

In [ ]:
wrapper_image = ImageData(arr)

In [ ]:
wrapper_image.data

In [ ]:
info = caiman_mc(ImageData(fnames[0]))

In [ ]:
caiman_cnmf(info['images'])